Исполнитель: Саломатин Антон Александрович

# Классификация названий спортивных школ

## Описание проекта
Сервис заказчика помогает спортивным школам фигурного катания и их тренерам мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.

### Цель проекта

Создать решение для стандартизации названий спортивных школ. Одна и та же школа может быть записана по-разному. Надо сопоставить эти варианты эталонному названию из предоставленной таблицы.

**Задачи**
- Создать модель для подбора наиболее вероятных названий при ошибочном вводе.
- Протестировать решение.

**Возможный стек**

python, pandas, scikit-learn, NLP, transformers


### Исходные данные

Исходные данные представлены двумя файлами формата ".csv":
- **Школы.csv** (обучающая выборка)
  - `school_id` - идентификатор школы;
  - `name` - название школы;
  - `region` - регион расположения школы.
<br><br>
- **Примерное написание.csv** (тестовая выборка)
  - `school_id` - идентификатор школы;
  - `name` - название школы.

### Этапы проекта

Решать поставленную задачу будем в следующем порядке:

- Загрузим и изучим данные.
- Создадим обучающий датасет.
- Подготовим решение по мэтчингу произвольных вариантов написания названий и эталонных.
- Дообучим модель.
- Выполним тестирование и сравним результаты исходной модели и дообученной.
- Сделаем описание решения, выводы и рекомендации.
    

## Загрузка и изучение данных

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import warnings
import nltk
import random

# Загружаем необходимый ресурс wordnet из NLTK
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')

# Из библиотек импортируем необходимые функции и классы
from textaugment import EDA # модуль для аугментации
from sentence_transformers import SentenceTransformer, util, InputExample, losses
from torch.utils.data import DataLoader
from tqdm import tqdm, trange
from datasets import Dataset
from fuzzywuzzy import fuzz, process

# Константы
# Количество наиболее подходящих вариантов, которые возвращает модель
TOP_K = 5

# Настройки
warnings.filterwarnings('ignore') # скрываем предупреждения

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\salom\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\salom\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\salom\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\salom\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\salom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Создадим датафреймы**

In [2]:
# Создадим датафрейм из файла "Школы.csv"
df_schools = pd.read_csv('gitignore/data/Школы.csv', index_col='school_id')

# Создадим датафрейм из файла "Примерное написание.csv"
df_example_input = pd.read_csv('gitignore/data/Примерное написание.csv', index_col='school_id')

**Изучим датафрейм со списком школ `df_schools`**

In [3]:
# выведем первые и последние строки датафрейма
df_schools

,name,region
school_id,,
1,Авангард,Московская область
2,Авангард,Ямало-Ненецкий АО
3,Авиатор,Республика Татарстан
4,Аврора,Санкт-Петербург
5,Ice Dream / Айс Дрим,Санкт-Петербург
...,...,...
305,Прогресс,Алтайский край
609,"""СШ ""Гвоздика""",Удмуртская республика
610,"СШОР ""Надежда Губернии",Саратовская область


In [4]:
# Выведем общую информацию
df_schools.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306 entries, 1 to 1836
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    306 non-null    object
 1   region  306 non-null    object
dtypes: object(2)
memory usage: 7.2+ KB


- Данные загружены корректно.
- Пропусков не обнаружено.
- Типы данных верны.
- Названия школ, как видно из первых строк, могут повторяться. 
- Необходимо проверить данные на наличие полных дубликатов.

**Проверим данные на явные дубликаты**

In [5]:
# Проверим df_schools на наличие дубликатов
df_schools.duplicated().sum()

0

In [6]:
# Проверим дубликаты в столбце 'name'
df_schools['name'].duplicated().sum()

43

In [7]:
# Проверим дубликаты в столбце 'region'
df_schools['region'].duplicated().sum()

232

In [8]:
# Проверим дубликаты попарно в столбцах 'name' + 'region'
df_schools[['name', 'region']].duplicated().sum()

0

**Проверим данные на неявные дубликаты. Объединим имя и регион в один столбец.**

In [9]:
df_schools['name_region'] = df_schools['name'] + ' ' + df_schools['region']
df_schools

,name,region,name_region
school_id,,,
1,Авангард,Московская область,Авангард Московская область
2,Авангард,Ямало-Ненецкий АО,Авангард Ямало-Ненецкий АО
3,Авиатор,Республика Татарстан,Авиатор Республика Татарстан
4,Аврора,Санкт-Петербург,Аврора Санкт-Петербург
5,Ice Dream / Айс Дрим,Санкт-Петербург,Ice Dream / Айс Дрим Санкт-Петербург
...,...,...,...
305,Прогресс,Алтайский край,Прогресс Алтайский край
609,"""СШ ""Гвоздика""",Удмуртская республика,"""СШ ""Гвоздика"" Удмуртская республика"
610,"СШОР ""Надежда Губернии",Саратовская область,"СШОР ""Надежда Губернии Саратовская область"


In [10]:
# Используем библиотеку fuzzywuzzy для поиска неявных дубликатов
# Функция для поиска похожих строк в датафрейме:
def find_similar_pairs(df, column, threshold=80):
    similar_pairs = []  # Список для хранения найденных пар
    seen_pairs = set()  # Множество для хранения уже найденных пар
    
    for i, row in df.iterrows():  # Проход по каждой строке датафрейма
        matches = process.extractBests(row[column], 
                                       df[column], 
                                       scorer=fuzz.token_sort_ratio, 
                                       score_cutoff=threshold)
        
        # Поиск наиболее похожих строк на основе порога схожести
        for match in matches:
            match_index = df[df[column] == match[0]].index[0]
            pair = tuple(sorted((i, match_index)))  # Нормализация пары
            
            if i != match_index and pair not in seen_pairs:  # Исключение сравнения строки с самой собой и проверка пары
                similar_pairs.append((i, row[column], match_index, match[0], match[1]))
                seen_pairs.add(pair)  # Добавление новой пары в множество
                
    return similar_pairs

In [11]:
# Поиск похожих пар строк в датафрейме
similar_pairs = find_similar_pairs(df_schools, 'name_region', threshold=90)

# Отображение результатов
for pair in similar_pairs:
    index_1, name_region_1, index_2, name_region_2, similarity = pair
    
    # Вывод пар строк с их регионами, индексами и коэффициентом схожести
    print(f'Pair: Index {index_1} \'{name_region_1}\' and Index {index_2} \'{name_region_2}\', Similarity: {similarity}')

Pair: Index 48 'Динамо Санкт-Петербург Санкт-Петербург' and Index 277 'НП КФК "Динамо-Санкт-Петербург" Санкт-Петербург', Similarity: 92
Pair: Index 93 'Маска Санкт-Петербург' and Index 273 'Каскад Санкт-Петербург', Similarity: 93
Pair: Index 97 'МАФСУ СШ № 6 Кемеровская область' and Index 98 'МАФСУ СШ №1 Кемеровская область', Similarity: 97
Pair: Index 97 'МАФСУ СШ № 6 Кемеровская область' and Index 100 'МБФСУ СШ Кемеровская область', Similarity: 93
Pair: Index 98 'МАФСУ СШ №1 Кемеровская область' and Index 100 'МБФСУ СШ Кемеровская область', Similarity: 93
Pair: Index 131 'Пируэт Санкт-Петербург' and Index 289 'ООО "Пируэт" Санкт-Петербург', Similarity: 92
Pair: Index 141 'РОФФКК Ростовская область' and Index 155 'СК РОФФКК Ростовская область', Similarity: 94
Pair: Index 153 'СиТ Республика Татарстан' and Index 172 'Стрела Республика Татарстан', Similarity: 90
Pair: Index 170 'Стартайс Санкт-Петербург' and Index 288 'СТАРТАЙС Санкт-Петербург', Similarity: 100
Pair: Index 179 'СШ по ФК

In [12]:
# Созданиим список индексов для удаления дубликатов (оставим только вторую строку пары)
indexes_to_remove = [48, 131, 141, 170, 186, 200, 213]

# Удаление дубликатов из датафрейма
df_schools = df_schools.drop(index=indexes_to_remove)

# Выведем первые строки df_schools
df_schools.head()

,name,region,name_region
school_id,,,
1,Авангард,Московская область,Авангард Московская область
2,Авангард,Ямало-Ненецкий АО,Авангард Ямало-Ненецкий АО
3,Авиатор,Республика Татарстан,Авиатор Республика Татарстан
4,Аврора,Санкт-Петербург,Аврора Санкт-Петербург
5,Ice Dream / Айс Дрим,Санкт-Петербург,Ice Dream / Айс Дрим Санкт-Петербург


In [13]:
# Выведем общую информацию о датафрейме df_schools
df_schools.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 1 to 1836
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         299 non-null    object
 1   region       299 non-null    object
 2   name_region  299 non-null    object
dtypes: object(3)
memory usage: 9.3+ KB


- Обнаружены дубликаты в столбцах `name` и `region`, но в парах `name` + `region` дубликатов не обнаружено.
- Обнаружены и удалены неявные дубликаты. Создан новый столбец `name_region` путем объединения данных из существующих.

**Изучим датафрейм с примерным написание школ `df_example_input`**

In [14]:
# выведем первые и последние строки датафрейма
df_example_input

,name
school_id,
1836,"ООО ""Триумф"""
1836,"Москва, СК ""Триумф"""
610,"СШОР ""Надежда Губернии"
610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
609,"""СШ ""Гвоздика"""
...,...
3,"Республика Татарстан, СШОР ФСО Авиатор"
3,"СШОР ФСО Авиатор, Республика Татарстан"
3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»"


In [15]:
# выведем общую информацию
df_example_input.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 895 entries, 1836 to 1
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    895 non-null    object
dtypes: object(1)
memory usage: 14.0+ KB


In [16]:
# Проверим на явные дубликаты датафрейм df_example_input
df_example_input.duplicated().sum()

1

In [17]:
# Удалим дубликат
df_example_input = df_example_input.drop_duplicates()

In [18]:
# выведем первые и последние строки датафрейма
df_example_input

,name
school_id,
1836,"ООО ""Триумф"""
1836,"Москва, СК ""Триумф"""
610,"СШОР ""Надежда Губернии"
610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
609,"""СШ ""Гвоздика"""
...,...
3,"Республика Татарстан, СШОР ФСО Авиатор"
3,"СШОР ФСО Авиатор, Республика Татарстан"
3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»"


In [19]:
# выведем общую информацию
df_example_input.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 894 entries, 1836 to 1
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    894 non-null    object
dtypes: object(1)
memory usage: 14.0+ KB


Удаление дубликата прошло корректно.

In [20]:
# Выведем статистики для уникальных значений индексов
df_example_input.index.value_counts().describe()

count    264.000000
mean       3.386364
std        3.139707
min        1.000000
25%        1.000000
50%        2.000000
75%        4.000000
max       19.000000
Name: school_id, dtype: float64

- Данные загружены корректно.
- Пропусков не обнаружено.
- Тип данных верный.
- Одному и тому же идентификатору школы могут соответствовать разные написания.
- Уникальных индексов: 264. Присутствует дисбаланс классов в тестовой выборке: количество объектов в каждом классе от 1 до 19.
- Обнаружен и удалён дубликат.

**Проверим все ли индексы `df_example_input` есть в `df_schools`.**

In [21]:
set1 = set(df_example_input.index.tolist())
set2 = set(df_schools.index.tolist())

# Преобразуем оба списка в множества и проверяем подмножество
is_subset = set1.issubset(set2)
print(is_subset)

False


In [22]:
# Найдем элементы, которые есть в set1, но отсутствуют в set2
missing_in_set2 = set1 - set2

# Найдем элементы, которые есть в set2, но отсутствуют в set1
missing_in_set1 = set2 - set1

print(f"Элементы из df_example_input, отсутствующие в df_schools: {missing_in_set2}")
print(f"Элементы из df_schools, отсутствующие в df_example_input: {missing_in_set1}")

Элементы из df_example_input, отсутствующие в df_schools: {131, 200, 48, 213, 186}
Элементы из df_schools, отсутствующие в df_example_input: {130, 134, 12, 268, 270, 271, 272, 145, 273, 153, 281, 283, 29, 285, 286, 288, 289, 39, 175, 305, 52, 57, 187, 68, 204, 77, 84, 218, 219, 220, 611, 230, 106, 107, 113, 114, 247, 120, 250, 127}


В тестовой выборке есть 5 классов, на которых модель не будет обучаться.

**Итог:**

- Данные заказчика загружены  в датафреймы: `df_schools` и `df_example_input`.
- Дубликаты удалены, их количество незначительно.
- `df_schools` имеет **299** записей, `df_example_input` имеет **894** записей.
- В тестовой выборке `df_example_input` присутствует дисбаланс классов, а также есть 5 неизвестных классов, на которых модель не будет обучаться. Изменять/исправлять тестовую выборку не будем, поскольку по данной выборке заказчик оценивает и сравнивает различные варианты решения данной задачи. Метрики качества модели будут объективны по отношению именно к данной выборке.

## Подготовка обучающей выборки

In [23]:
df_schools

,name,region,name_region
school_id,,,
1,Авангард,Московская область,Авангард Московская область
2,Авангард,Ямало-Ненецкий АО,Авангард Ямало-Ненецкий АО
3,Авиатор,Республика Татарстан,Авиатор Республика Татарстан
4,Аврора,Санкт-Петербург,Аврора Санкт-Петербург
5,Ice Dream / Айс Дрим,Санкт-Петербург,Ice Dream / Айс Дрим Санкт-Петербург
...,...,...,...
305,Прогресс,Алтайский край,Прогресс Алтайский край
609,"""СШ ""Гвоздика""",Удмуртская республика,"""СШ ""Гвоздика"" Удмуртская республика"
610,"СШОР ""Надежда Губернии",Саратовская область,"СШОР ""Надежда Губернии Саратовская область"


**1. Очистим текст в столбце 'name_region' от ненужных знаков вроде кавычек**

In [24]:
# Создадим функцию для очистки названий школ от лишних символов
def clean_names(col):
    return (col.replace(r'\(.+\)', ' ', regex=True)                  # замена содержимого скобок на пробел
               .replace(r'[^А-Яа-яёЁA-Za-z0-9\s]', ' ', regex=True)  # замена на пробел всех символов, кроме рус. и англ. букв, цифр и пробелов
               .replace(r'\s+', ' ', regex=True)                     # замена нескольких пробелов на один
               .str.strip()                                          # удаление лишних пробелов в начале и конце строки
               .str.lower()                                          # приведение всей строки к нижнему регистру
           )                                         

In [25]:
# очистим названия школ от лишних символов
df_schools['clean_name_region'] = clean_names(df_schools['name_region'])

# выведем срез, где есть цифры
df_schools[50:60]

,name,region,name_region,clean_name_region
school_id,,,,
52,ДЮСШ СК Юбилейный,Санкт-Петербург,ДЮСШ СК Юбилейный Санкт-Петербург,дюсш ск юбилейный санкт петербург
53,ДЮСШ No 1,Тюменская область,ДЮСШ No 1 Тюменская область,дюсш no 1 тюменская область
54,ДЮСШ № 10,Самарская область,ДЮСШ № 10 Самарская область,дюсш 10 самарская область
55,ДЮСШ № 2,Курганская область,ДЮСШ № 2 Курганская область,дюсш 2 курганская область
56,ДЮСШ № 2,Республика Карелия,ДЮСШ № 2 Республика Карелия,дюсш 2 республика карелия
57,ДЮСШ № 4,Санкт-Петербург,ДЮСШ № 4 Санкт-Петербург,дюсш 4 санкт петербург
58,ДЮСШ № 7,Иркутская область,ДЮСШ № 7 Иркутская область,дюсш 7 иркутская область
59,ДЮСШ № 8,Владимирская область,ДЮСШ № 8 Владимирская область,дюсш 8 владимирская область
60,Ермак,Иркутская область,Ермак Иркутская область,ермак иркутская область


**2. Выполним аугментацию**

In [26]:
# инициализируем модуль для расширенной аугментации
t = EDA()

In [27]:
# Словарь с соседними клавишами для русской клавиатуры
keyboard_adjacent_keys = {
    'й': 'цу', 'ц': 'уий', 'у': 'йцык', 'к': 'еуцу', 'е': 'укн', 'н': 'гшен', 'г': 'ро', 'ш': 'нгг', 'щ': 'кнг',
    'з': 'гшщ', 'х': 'шщз', 'ъ': 'аз', 'ф': 'ыва', 'ы': 'выаф', 'в': 'фаыпр', 'а': 'ифва', 'п': 'рлв', 'р': 'коп',
    'о': 'ете', 'л': 'дожр', 'д': 'ла', 'ж': 'олд', 'э': 'жэ', 'я': 'чта', 'ч': 'ся', 'с': 'чь', 'м': 'тить',
    'и': 'тнь', 'т': 'итьв', 'ь': 'бси', 'б': 'юь', 'ю': 'юй'
}

# Словарь со спортивными школами и их аббревиатурами
sports_schools = {
    "Детско-юношеская спортивная школа": "ДЮСШ",
    "Специализированная детско-юношеская спортивная школа олимпийского резерва": "СДЮСШОР",
    "Спортивная школа олимпийского резерва": "СШОР",
    "Школа высшего спортивного мастерства": "ШВСМ",
    "Училище олимпийского резерва": "УОР",
    "Центр олимпийской подготовки": "ЦОП",
    "Центр спортивной подготовки": "ЦСП",
    "Школа интернат спортивного профиля": "ШИСП",
    "Академия спорта": "АС",
    "Спортивно-оздоровительный комплекс": "СОК",
    "Спортивный клуб": "СК",
    "Федерация спортивной гимнастики": "ФСГ",
    "Федерация футбола": "ФФ",
    "Федерация легкой атлетики": "ФЛА",
    "Федерация бокса": "ФБ",
    "Спортивная детско-юношеская школа": "СДЮШ",
    "Комплексная спортивная школа": "КСШ",
    "Областная спортивная школа": "ОСШ",
    "Олимпийский центр подготовки": "ОЦП",
    "Спорткомплекс": "СК",
    "Федеральный центр подготовки спортивного резерва": "ФЦПСР",
    "Спортивная школа интернат": "ШШИ",
    "Спортивная секция": "СС",
    "Детский спортивный клуб": "ДСК",
    "Городская спортивная школа": "ГСШ",
    "Региональный центр спортивной подготовки молодежи": "РЦСПМ",
    "Училище физической культуры": "УФК",
    "Национальная спортивная школа": "НСШ",
    "Муниципальная спортивная школа": "МСШ",
    "Спортивная школа боевых искусств": "ШШБИ",
    "Центр подготовки олимпийского резерва": "ЦПОР",
    "Международная спортивная академия": "МСА",
    "Центр детского спорта": "ЦДС",
    "Комбинат тренировочных залов": "КТЗ",
    "Спортивная школа": "ШШ",
    "Областной центр развития одаренных спортсменов": "ОЦРОС",
    "Центр обучения олимпийских резервов": "ЦОО",
    "Спортивная школа олимпийского резерва на базе школьной инфраструктуры": "ШШОРШИ",
    "Академический спортивный центр": "АСЦ",
    "Государственный центр подготовки спортивного резерва": "ГЦПСР",
    "Центральная школа олимпийского резерва": "ЦШОР",
    "Государственная спортивная школа": "ГСШ",
    "Спортивный колледж": "СК",
    "Центр физической культуры и спорта": "ЦФКС",
    "Федерация шахмат": "ФШ",
    "Федерация плавания": "ФП",
    "Федерация волейбола": "ФВБ",
    "Федерация настольного тенниса": "ФНТ",
    "Региональная спортивная школа": "РШ",
    "Спортивная школа-интернат олимпийского резерва": "ШШИОР"
}

# Словарь с аббревиатурой городов
cities = {
    "Москва": "МСК",
    "Санкт-Петербург": "СПБ",
    "Новосибирск": "НСБ",
    "Екатеринбург": "ЕКБ",
    "Нижний Новгород": "НН",
    "Казань": "КЗН",
    "Челябинск": "ЧЛБ",
    "Омск": "ОМС",
    "Ростов-на-Дону": "РНД",
    "Уфа": "УФА",
    "Красноярск": "КРС",
    "Пермь": "ПРМ",
    "Волгоград": "ВЛГ",
    "Воронеж": "ВРН",
    "Саратов": "САР",
    "Краснодар": "КРД",
    "Тольятти": "ТЛТ",
    "Барнаул": "БРН",
    "Ижевск": "ИЖ",
    "Ульяновск": "УЛЬ",
    "Владивосток": "ВЛД",
    "Ярославль": "ЯРОСЛАВЛЬ",
    "Иркутск": "ИРК",
    "Тюмень": "ТЮМ",
    "Махачкала": "МХЧ",
    "Оренбург": "ОРН",
    "Новокузнецк": "НОК",
    "Кемерово": "КЕМ",
    "Рязань": "РЗН",
    "Астрахань": "АСТ",
    "Набережные Челны": "НЧЛ",
    "Пенза": "ПЕН",
    "Липецк": "ЛИП",
    "Тула": "ТУЛ",
    "Киров": "КИР",
    "Чебоксары": "ЧБК",
    "Курск": "КУР",
    "Брянск": "БРН",
    "Магнитогорск": "МГТ",
    "Иваново": "ИВН",
    "Тверь": "ТВР",
    "Ставрополь": "СТВ",
    "Симферополь": "СИМ",
    "Белгород": "БЛГ",
    "Сочи": "СОЧ",
    "Калуга": "КАЛ",
    "Смоленск": "СМЛ",
    "Владимир": "ВЛДМ",
    "Архангельск": "АРХ",
    "Сургут": "СУР",
    "Чита": "ЧИТ",
    "Саранск": "САРР",
    "Вологда": "ВЛГД",
    "Тамбов": "ТМБ",
    "Грозный": "ГРЗ",
    "Якутск": "ЯКТ",
    "Кострома": "КСТ",
    "Петропавловск-Камчатский": "ПК",
    "Йошкар-Ола": "ЙО",
    "Хабаровск": "ХБР",
    "Анапа": "АНА",
    "Псков": "ПСК",
    "Орехово-Зуево": "ОЗ",
    "Калининград": "КЛОН",
    "Абакан": "АБК",
    "Крымск": "КРМ",
    "Сызрань": "СЫЗ",
    "Минеральные Воды": "МВ",
    "Нижнекамск": "НЖК",
    "Северск": "СЕВ",
    "Севастополь": "СЕВСТ",
    "Алупка": "АЛП",
    "Железногорск": "ЖЛЗ"
}

# Словарь с аббревиатурами регионов
regions = {
    "Республика Адыгея": "АД",
    "Республика Алтай": "АЛТ",
    "Республика Башкортостан": "БАШ",
    "Республика Бурятия": "БУР",
    "Республика Дагестан": "ДАГ",
    "Республика Ингушетия": "ИНГ",
    "Кабардино-Балкарская Республика": "КБР",
    "Республика Калмыкия": "КАЛМ",
    "Карачаево-Черкесская Республика": "КЧР",
    "Республика Карелия": "КАР",
    "Республика Коми": "КОМ",
    "Республика Крым": "КРЫМ",
    "Республика Марий Эл": "МАР",
    "Республика Мордовия": "МОР",
    "Республика Саха (Якутия)": "САХА",
    "Республика Северная Осетия - Алания": "СОА",
    "Республика Татарстан": "ТАТ",
    "Республика Тыва": "ТЫВА",
    "Удмуртская Республика": "УДМ",
    "Республика Хакасия": "ХАК",
    "Чеченская Республика": "ЧЕЧ",
    "Чувашская Республика": "ЧУВ",
    "Алтайский край": "АЛТК",
    "Забайкальский край": "ЗАБК",
    "Камчатский край": "КМЧ",
    "Краснодарский край": "КРД",
    "Красноярский край": "КРС",
    "Пермский край": "ПЕРМ",
    "Приморский край": "ПРМ",
    "Ставропольский край": "СТВ",
    "Хабаровский край": "ХБР",
    "Амурская область": "АМУР",
    "Архангельская область": "АРХ",
    "Астраханская область": "АСТР",
    "Белгородская область": "БЕЛ",
    "Брянская область": "БРЯН",
    "Владимирская область": "ВЛД",
    "Волгоградская область": "ВОЛГ",
    "Вологодская область": "ВОЛГД",
    "Воронежская область": "ВРН",
    "Ивановская область": "ИВАН",
    "Иркутская область": "ИРК",
    "Калининградская область": "КАЛ",
    "Калужская область": "КАЛУ",
    "Кемеровская область": "КЕМ",
    "Кировская область": "КИР",
    "Костромская область": "КОСТ",
    "Курганская область": "КУРГ",
    "Курская область": "КУР",
    "Ленинградская область": "ЛЕН",
    "Липецкая область": "ЛИП",
    "Магаданская область": "МГД",
    "Московская область": "МОСК",
    "Мурманская область": "МУР",
    "Нижегородская область": "НИЖ",
    "Новгородская область": "НОВГ",
    "Новосибирская область": "НСО",
    "Омская область": "ОМС",
    "Оренбургская область": "ОРН",
    "Орловская область": "ОРЛ",
    "Пензенская область": "ПЕН",
    "Псковская область": "ПСК",
    "Ростовская область": "РОСТ",
    "Рязанская область": "РЯЗ",
    "Самарская область": "САМ",
    "Саратовская область": "САР",
    "Сахалинская область": "САХ",
    "Свердловская область": "СВЕРД",
    "Смоленская область": "СМОЛ",
    "Тамбовская область": "ТАМБ",
    "Тверская область": "ТВЕР",
    "Томская область": "ТОМ",
    "Тульская область": "ТУЛ",
    "Тюменская область": "ТЮМ",
    "Ульяновская область": "УЛЬЯН",
    "Челябинская область": "ЧЕЛ",
    "Ярославская область": "ЯРОС",
    "Москва": "МСК",
    "Санкт-Петербург": "СПБ",
    "Еврейская автономная область": "ЕАО",
    "Ненецкий автономный округ": "НЕН",
    "Ханты-Мансийский автономный округ – Югра": "ХМАО",
    "Чукотский автономный округ": "ЧАО",
    "Ямало-Ненецкий автономный округ": "ЯНАО"
}

# Перезапись словарей с ключами и значениями в нижнем регистре длд повышения точности классификации
lowercase_sports_schools = {k.lower(): v.lower() for k, v in sports_schools.items()}
cities = {k.lower(): v.lower() for k, v in cities.items()}
regions = {k.lower(): v.lower() for k, v in regions.items()}

In [28]:
# Эта функция вставляет опечатку в заданное слово.
def insert_typo(word):
    if len(word) > 2 and not word.isdigit():  # Только если слово длинее 2 символов и не является числом
        pos = random.randint(0, len(word) - 1)  # Выбираем случайную позицию в слове
        original_char = word[pos]
        if original_char in keyboard_adjacent_keys:  # Проверяем, есть ли символ в словаре соседних кнопок
            typo_char = random.choice(keyboard_adjacent_keys[original_char])  # Выбираем случайную соседнюю кнопку
            return word[:pos] + typo_char + word[pos + 1:]  # Вставляем опечатку в слово
        else:
            return word  # Если символа нет в словаре, оставляем слово без изменений
    return word  # Для слов короче 2 символов или чисел возвращаем слово без изменений

# Эта функция создает опечатки в одном слове строки.
def create_typo(text):
    words = text.split()  # Разделяем текст на слова
    if words:  # Проверяем, что список слов не пуст
        word_index = random.randint(0, len(words)-1)
        words[word_index] = insert_typo(words[word_index])  # Для выбранного слова создаем опечатку
    return " ".join(words)  # Объединяем слова обратно в строку

# Функция для замены аббревиатур на полные названия в тексте
def replace_with_full_names(text, dictionary):
    for abbr, full in dictionary.items():
        text = text.replace(abbr, full)  # Заменяем все вхождения аббревиатуры на полное название
    return text

# Функция для замены полных названий на аббревиатуры в тексте
def replace_with_abbreviations(text, dictionary):
    for full, abbr in dictionary.items():
        text = text.replace(full, abbr)  # Заменяем все вхождения полного названия на аббревиатуру
    return text

# Функция для удаления случайных слов
def random_deletion(text, p=0.2):
    words = text.split()
    if len(words) == 1:  # Если одно слово, не удаляем его
        return text

    remaining = list(filter(lambda x: random.uniform(0, 1) > p, words))
    if len(remaining) == 0:  # Чтобы не вернуть пустую строку
        return ' '.join(random.sample(words, k=random.randint(1, len(words))))
    else:
        return ' '.join(remaining)

# Функция для случайного перемешивания слов
def random_swap(text, n=1):
    words = text.split()
    if len(words) < 2:
        return text  # Возвращаем исходный текст, если слов меньше двух
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return ' '.join(words)

# функцию для симуляции пропущенной буквы в слове 
def simulate_typo_missing_letter(word):
    if len(word) > 1:  # Только если слово длинее 1 символа
        pos = random.randint(0, len(word) - 1)  # Выбираем случайную позицию в слове
        return word[:pos] + word[pos + 1:]  # Удаляем символ из слова
    return word  # Для коротких слов возвращаем слово без изменений

**3. Сформируем обучающую выборку**

In [29]:
%%time
def augment_text(original_text, n_augmentations):
    augmentations = set()
    attempt_count = 0 # количество попыток
    augmentations.add(original_text)
    
    aug_methods = [
        lambda text: replace_with_full_names(text, cities),            # замена на аббр. города
        lambda text: replace_with_full_names(text, sports_schools),    # замена на аббр. названия школ
        lambda text: replace_with_full_names(text, regions),           # замена на аббр. регионы
        lambda text: replace_with_abbreviations(text, cities),         # замена аббр. городов
        lambda text: replace_with_abbreviations(text, sports_schools), # замена аббр. названий школ
        lambda text: replace_with_abbreviations(text, regions),        # замена аббр. регионов
        random_deletion,                                               # удаление случайного слова
        random_swap,                                                   # случайное перемешивание слов
        create_typo,                                                   # делает опечатку в слове
        
        # удаляет букву в слове
        lambda txt: " ".join([simulate_typo_missing_letter(word) for word in txt.split()])
    ]

    for i in range(len(aug_methods)):
        while len(augmentations) < n_augmentations + 1:
            before = len(augmentations)
            method = aug_methods[i]
            new_text = method(original_text)
            augmentations.add(new_text)
            
            if len(augmentations) <= before:
                break
    augmentations.remove(original_text)
    return list(augmentations)


# Создаем новый DataFrame для хранения всех аугментаций
train = pd.DataFrame(columns=['original_text', 'augmented_text'])

# Итеративно применяем аугментации к каждому элементу DataFrame
for index, row in df_schools.iterrows():
    original_text = row['clean_name_region']
    augmented_texts = augment_text(original_text, n_augmentations=15)

    temp_df = pd.DataFrame({
        'original_text': [original_text] * len(augmented_texts),
        'augmented_text': augmented_texts
    })

    train = pd.concat([train, temp_df], ignore_index=True)
print(train.shape[0])
train[:20]

4428
CPU times: total: 312 ms
Wall time: 312 ms


,original_text,augmented_text
0,авангард московская область,авагард московкая облась
1,авангард московская область,вангард московкая облась
2,авангард московская область,вангард осковская облась
3,авангард московская область,авагард москоская оласть
4,авангард московская область,вангард московскя оласть
5,авангард московская область,аангард москоская бласть
6,авангард московская область,авангард область московская
7,авангард московская область,авангард моск
8,авангард московская область,авангард московская областб
9,авангард московская область,авагард московсая обасть


**Итог:**

Обучающий датасет `train` готов. При необходимости его можно увеличить регулируя `n_augmentations` для повышения точности обученной модели, если имеются достаточно мощные вычислительные ресурсы.

## Подготовка тестовой выборки

In [30]:
# тестовую выборку формируем на основе df_example_input
test = df_example_input.copy()

# очистим имена от ненужных знаков и выненем в новый столбец
test['clean_name'] = clean_names(test['name'])

test

,name,clean_name
school_id,,
1836,"ООО ""Триумф""",ооо триумф
1836,"Москва, СК ""Триумф""",москва ск триумф
610,"СШОР ""Надежда Губернии",сшор надежда губернии
610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...",саратовская область гбусо сшор надежда губернии
609,"""СШ ""Гвоздика""",сш гвоздика
...,...,...
3,"Республика Татарстан, СШОР ФСО Авиатор",республика татарстан сшор фсо авиатор
3,"СШОР ФСО Авиатор, Республика Татарстан",сшор фсо авиатор республика татарстан
3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»",республика татарстан мбу до сшор фсо авиатор


**Итог:**

- Тестовая выборка готова для анализа. Для более точных результатов будет использован очищенный и подготовленный столбец `clean_name`.

## Получение результатов необученной модели

In [31]:
# инициализируем модель LaBSE
model = SentenceTransformer('sentence-transformers/LaBSE')

In [32]:
# Создаём эмбеддинги для всех названий школ (очищенных) из списка df_schools 
corpus = model.encode(df_schools.clean_name_region.values)
corpus

array([[ 0.0422793 ,  0.02789922,  0.02638609, ..., -0.03887581,
        -0.06346421, -0.05755831],
       [-0.00865452,  0.00656926,  0.02861418, ..., -0.00812264,
        -0.08503363, -0.03980788],
       [ 0.01162196, -0.03974552,  0.00175535, ...,  0.02332449,
         0.06898224,  0.0510359 ],
       ...,
       [ 0.04054917, -0.04491944,  0.04283404, ...,  0.04358646,
         0.0482295 , -0.02865027],
       [ 0.05014742, -0.01094766, -0.02868276, ..., -0.00640665,
         0.02301885,  0.00388744],
       [-0.02941606, -0.03230731, -0.00201937, ..., -0.0453206 ,
        -0.06100004, -0.04871305]], dtype=float32)

In [33]:
# Создаём эмбеддинги для всех названий школ (очищенных) из тестовой выборки 
query = model.encode(test.clean_name.values)
query

array([[-0.03568289, -0.04341515, -0.01187238, ..., -0.00016677,
        -0.06623294, -0.0480037 ],
       [-0.00549194, -0.02082244,  0.00669351, ..., -0.0385594 ,
        -0.00908837, -0.02304343],
       [ 0.01332995,  0.00126805,  0.02289892, ...,  0.06589615,
         0.01853497, -0.0119269 ],
       ...,
       [ 0.01080701, -0.01810541, -0.00604198, ...,  0.06872355,
         0.05091148,  0.07889753],
       [ 0.00098638, -0.0160746 , -0.01331254, ...,  0.01204279,
        -0.06677934, -0.05498999],
       [ 0.03829782,  0.01383218,  0.01393366, ..., -0.01315422,
        -0.04395086, -0.04659519]], dtype=float32)

In [34]:
# Принимаем на вход эмбеддинг для запроса, эмбеддинг для корпуса, и параметр сколько кандидатов надо возвращать
search_result = util.semantic_search(query, corpus, top_k=TOP_K)


# Преобразование результатов поиска с использованием пользовательских индексов, названий и значений score
def search_results():
    # Словарь для соответствия эмбеддингов и пользовательских индексов
    # Эмбеддинги нумеруются, что не соответствуют индексам в df_schools
    index_map = {i: df_schools.index[i] for i in range(len(df_schools))}
    
    
    real_index_search_result = []
    score_search_result = []
    name_search_result = []
    
    # Проходим по каждому результату поиска
    for result in search_result:
        real_index_result = []  # список реальных индексов для текущего результата поиска
        score_result = []       # список оценок соответствия для текущего результата поиска
        name_result = []        # список имен кандидатов для текущего результата поиска
    
        # Проходим по каждому совпадению в текущем результате поиска
        for r in result:
            idx = index_map[r['corpus_id']]  # получаем реальный индекс из словаря index_map
            real_index_result.append(idx)    # добавляем реальный индекс в список
            score_result.append(round(r['score'],2)) # добавляем оценку соответствия в список
            name_result.append(df_schools.loc[idx, 'name'])  # добавляем имя кандидата в список
    
        # Добавляем списки реальных индексов, оценок и имен кандидатов в соответствующие итоговые списки
        real_index_search_result.append(real_index_result)
        score_search_result.append(score_result)
        name_search_result.append(name_result)
    
    # Добавляем новые столбцы в DataFrame test
    # candidate_idx - реальные индексы кандидатов
    # candidate_scores - оценки соответствия кандидатов
    # candidate_names - имена кандидатов
    test['candidate_idx'] = real_index_search_result
    test['candidate_scores'] = score_search_result
    test['candidate_names'] = name_search_result
    
    return test

# добавляем результаты в тестовую таблицу
test = search_results()
test

,name,clean_name,candidate_idx,candidate_scores,candidate_names
school_id,,,,,
1836,"ООО ""Триумф""",ооо триумф,"[1836, 225, 173, 123, 265]","[0.77, 0.49, 0.47, 0.45, 0.4]","[ООО ""Триумф"", Триумф, СШ, Олимпиец, Юность]"
1836,"Москва, СК ""Триумф""",москва ск триумф,"[1836, 104, 251, 136, 96]","[0.79, 0.7, 0.7, 0.68, 0.67]","[ООО ""Триумф"", МОСГОРСПОРТ, ЦСКА, ПроСинхро, М..."
610,"СШОР ""Надежда Губернии",сшор надежда губернии,"[610, 110, 175, 193, 32]","[0.71, 0.54, 0.51, 0.49, 0.49]","[СШОР ""Надежда Губернии, Наши надежды, СШ ЗВС,..."
610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...",саратовская область гбусо сшор надежда губернии,"[610, 40, 216, 54, 175]","[0.94, 0.6, 0.59, 0.59, 0.58]","[СШОР ""Надежда Губернии, Голубева, СШОР № 6, Д..."
609,"""СШ ""Гвоздика""",сш гвоздика,"[173, 609, 251, 138, 252]","[0.5, 0.48, 0.45, 0.41, 0.4]","[СШ, ""СШ ""Гвоздика"", ЦСКА, Прусова, ЦСКА им. С..."
...,...,...,...,...,...
3,"Республика Татарстан, СШОР ФСО Авиатор",республика татарстан сшор фсо авиатор,"[3, 201, 143, 230, 130]","[0.86, 0.75, 0.74, 0.73, 0.69]","[Авиатор, СШОР по ФККиШТ, РСШОР, ФАУ МО РФ ЦСК..."
3,"СШОР ФСО Авиатор, Республика Татарстан",сшор фсо авиатор республика татарстан,"[3, 201, 143, 230, 130]","[0.82, 0.74, 0.71, 0.7, 0.65]","[Авиатор, СШОР по ФККиШТ, РСШОР, ФАУ МО РФ ЦСК..."
3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»",республика татарстан мбу до сшор фсо авиатор,"[3, 201, 230, 143, 130]","[0.78, 0.73, 0.71, 0.7, 0.65]","[Авиатор, СШОР по ФККиШТ, ФАУ МО РФ ЦСКА, РСШО..."


In [35]:
# Функция для подсчета случаев, когда school_id равен первому значению candidate_idx
def count_first_occurrences(row):
    return row.name == row['candidate_idx'][0] if row['candidate_idx'] else False

# Функция для подсчета случаев, когда school_id равен любому значению candidate_idx
def count_any_occurrences(row):
    return row.name in row['candidate_idx']

# Применяем функции и вычислим результаты для всей таблицы
count_first = test.apply(count_first_occurrences, axis=1).sum()
count_any = test.apply(count_any_occurrences, axis=1).sum()

# точность прогноза
print(f'Размер обучающей выборки: {train.shape[0]}')
print(f'accuracy_at_1 = {count_first / test.shape[0]} (точность первого варианта)')
print(f'accuracy_at_{TOP_K} = {count_any / test.shape[0]}  (точность первых пяти вариантов)')

Размер обучающей выборки: 4428
accuracy_at_1 = 0.7483221476510067 (точность первого варианта)
accuracy_at_5 = 0.8859060402684564  (точность первых пяти вариантов)


**Вывод:**

- Необученная модель позволяет достаточно точно найти 5 вариантов, к которым относится рассматриваемое название школы. 
- Модель также позволяет подобрать такой параметр `score`, при котором практически гарантированно часть названий будет точно идентифицирована. Это сократит дальнейшую ручную проверку.

## Дообучение модели

In [36]:
# дообучим модель на тренировочных данных
make_example = lambda x: InputExample(texts=[x['original_text'], x['augmented_text']])

examples = train[['original_text', 'augmented_text']].apply(make_example, axis=1).values

# Попытка подбора размера батча
max_batch_size = 64
found = False

while not found:
    try:
        train_dataloader = DataLoader(examples, shuffle=True, batch_size=max_batch_size)
        train_loss = losses.MultipleNegativesRankingLoss(model=model)
        model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2)
        found = True
    except RuntimeError as e:
        if 'out of memory' in str(e).lower():
            max_batch_size = max_batch_size // 2
            print(f"Уменьшение размера батча до {max_batch_size} из-за ошибки: {e}")
            if max_batch_size == 0:
                raise RuntimeError("Недостаточно памяти для выполнения с минимальным размером батча.")
        else:
            raise e

print("Модель успешно обучена с батчем: ", max_batch_size)

Step,Training Loss


Модель успешно обучена с батчем:  64


In [37]:
# тестовую перезаписываем на основе df_example_input
test = df_example_input.copy()

# очистим имена от ненужных знаков и выненем в новый столбец
test['clean_name'] = clean_names(test['name'])

In [38]:
# Создаём эмбеддинги для всех названий школ (очищенных) из списка df_schools 
corpus = model.encode(df_schools.clean_name_region.values)
corpus

array([[ 0.03794705,  0.02708246,  0.02707191, ..., -0.03725176,
        -0.06768388, -0.06156914],
       [-0.0143672 ,  0.00205906,  0.02572316, ..., -0.01239368,
        -0.08814749, -0.0416766 ],
       [ 0.01092311, -0.04225736, -0.00069241, ...,  0.02504339,
         0.06681807,  0.04724994],
       ...,
       [ 0.03280314, -0.04593975,  0.03921307, ...,  0.05103117,
         0.04452779, -0.02948497],
       [ 0.04858021, -0.01274694, -0.03533774, ..., -0.00435461,
         0.01932614,  0.0074805 ],
       [-0.0291154 , -0.03221771, -0.0021017 , ..., -0.04342784,
        -0.06397883, -0.04800384]], dtype=float32)

In [39]:
# Создаём эмбеддинги для всех названий школ (очищенных) из тестовой выборки 
query = model.encode(test.clean_name.values)
query

array([[-0.03552034, -0.04362733, -0.01121973, ..., -0.00015342,
        -0.06690256, -0.04770898],
       [-0.0034834 , -0.02224001,  0.00412947, ..., -0.03753417,
        -0.00966401, -0.0211281 ],
       [ 0.0051846 , -0.00227411,  0.01928673, ...,  0.06793853,
         0.01229474, -0.01449553],
       ...,
       [ 0.00950011, -0.01925739, -0.0100242 , ...,  0.07130997,
         0.04707031,  0.07847561],
       [-0.00569882, -0.02029031, -0.01052831, ...,  0.00961516,
        -0.06518649, -0.05850585],
       [ 0.03351078,  0.01271828,  0.0125618 , ..., -0.01128302,
        -0.04588163, -0.04978176]], dtype=float32)

In [40]:
# Принимаем на вход эмбеддинг для запроса, эмбеддинг для корпуса, и параметр сколько кандидатов надо возвращать
search_result = util.semantic_search(query, corpus, top_k=TOP_K)

# Добавляем результаты в тестовую таблицу
test = search_results()
test

,name,clean_name,candidate_idx,candidate_scores,candidate_names
school_id,,,,,
1836,"ООО ""Триумф""",ооо триумф,"[1836, 225, 173, 123, 289]","[0.79, 0.51, 0.48, 0.47, 0.44]","[ООО ""Триумф"", Триумф, СШ, Олимпиец, ООО ""Пиру..."
1836,"Москва, СК ""Триумф""",москва ск триумф,"[1836, 104, 251, 136, 96]","[0.78, 0.69, 0.69, 0.66, 0.66]","[ООО ""Триумф"", МОСГОРСПОРТ, ЦСКА, ПроСинхро, М..."
610,"СШОР ""Надежда Губернии",сшор надежда губернии,"[610, 110, 175, 193, 32]","[0.74, 0.54, 0.51, 0.51, 0.49]","[СШОР ""Надежда Губернии, Наши надежды, СШ ЗВС,..."
610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...",саратовская область гбусо сшор надежда губернии,"[610, 40, 216, 175, 54]","[0.92, 0.58, 0.56, 0.55, 0.55]","[СШОР ""Надежда Губернии, Голубева, СШОР № 6, С..."
609,"""СШ ""Гвоздика""",сш гвоздика,"[609, 173, 251, 252, 138]","[0.54, 0.52, 0.48, 0.44, 0.42]","[""СШ ""Гвоздика"", СШ, ЦСКА, ЦСКА им. С.А. Жука,..."
...,...,...,...,...,...
3,"Республика Татарстан, СШОР ФСО Авиатор",республика татарстан сшор фсо авиатор,"[3, 201, 143, 230, 130]","[0.83, 0.75, 0.74, 0.72, 0.67]","[Авиатор, СШОР по ФККиШТ, РСШОР, ФАУ МО РФ ЦСК..."
3,"СШОР ФСО Авиатор, Республика Татарстан",сшор фсо авиатор республика татарстан,"[3, 201, 143, 230, 130]","[0.78, 0.73, 0.7, 0.68, 0.62]","[Авиатор, СШОР по ФККиШТ, РСШОР, ФАУ МО РФ ЦСК..."
3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»",республика татарстан мбу до сшор фсо авиатор,"[3, 201, 230, 143, 130]","[0.75, 0.72, 0.7, 0.69, 0.62]","[Авиатор, СШОР по ФККиШТ, ФАУ МО РФ ЦСКА, РСШО..."


In [41]:
# Применяем функции и вычислим результаты для всей таблицы
count_first = test.apply(count_first_occurrences, axis=1).sum()
count_any = test.apply(count_any_occurrences, axis=1).sum()

# точность прогноза
print(f'accuracy_at_1 = {count_first / test.shape[0]} (точность первого варианта)')
print(f'accuracy_at_{TOP_K} = {count_any / test.shape[0]}  (точность первых пяти вариантов)')

accuracy_at_1 = 0.7583892617449665 (точность первого варианта)
accuracy_at_5 = 0.8993288590604027  (точность первых пяти вариантов)


**Итог:**

- Дообучение модели улучшило результаты незначительно. 
- Для более точный результатов имеет смысл протестировать другие методы аугментации, в т.ч. ислользуя готовые библиотеки. 
- В случае недостаточного эффекта при изменении аугментаций, нужно попробовать другие трансформеры. Тесты лучше производить, используя GPU для возможности обработки большого обучающего датасета.

## Общий вывод

**В данном проекте было необходимо решить следующую задачу**:

- Создать модель, которая будет подсказывать правильное наименование спортивной школы при ошибочном вводе.

**В ходе проекта мы провели следующую работу:**
- Изучили данные, проверили их на предмет аномалий и несоответствий.

- Протестировали необученную на наших данных модель (трансформер) LaBSE.

- Дообучили модель LaBSE и протестировали.

**Результаты:**

- Наилучшие результаты показала модель дообученная модель LaBSE: 
  - accuracy_at_1 = 0.76 (точность первого варианта)
  - accuracy_at_5 = 0.90  (точность первых пяти вариантов)
  
Но эти результаты несущественно отличаются от результатов необученной модель (accuracy_at_1 = 0.75, accuracy_at_5 = 0.89).

Обучение выполнялось на 8-ядерном CPU с весьма скромной производительностью, но тесты произведенные в облаке Google Colaboratory на GPU с числом строк обучающей выборки до 10 тыс. при обучении до 5 эпох включительно также не дали существенного прироста.

**Рекомендации:**

Необученную модель LaBSE можно использовать для точной классификации части введеных строк (можно подобрать приемлемое значение score, при котором с высокой вероятностью прогноз будет точным). Также можно подобрать score для 5 результатов, а остальные значения классифицировать вручную.

Для более точных прогнозов необходимо следующее:
1. Протестировать другие модели, используя многоядерную систему GPU и выбрать лучшую модель
2. Для лучшей модели протестировать разные принципы создания эмбеддингов, рассмотреть готовые библиотеки
